In [99]:
#imports

from gensim.models import Word2Vec
from gensim.similarities.index import AnnoyIndexer
import gensim

import re
import string
import numpy as np

import nltk
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist

from gensim.test.utils import common_texts
from gensim.models import FastText
from twitterscraper import query_tweets
import json
from collections import defaultdict
import datetime as dt
from langdetect import detect

In [145]:
#settings

dump_file = 'twitter_dump.json'
query = ['lacoste']
timeline_file = 'product_timeline.json'
chosen_dir = '/home/ruan/w2v_bd/'
start = dt.date(2018,1,1)

In [101]:

dct = defaultdict(list)

with open(dump_file, "w+") as f:
    json.dump(dct, f)

for q in query:
    for tweet in query_tweets(q, limit=10**5, begindate=start):
        try:
            if detect(tweet.text) == 'pt':
                unit = tweet.timestamp.month
                dct[unit].append(tweet.text)
        except:
            continue

with open(dump_file,"w+") as f:
    json.dump(dct, f)

INFO: queries: ['lacoste since:2018-01-01 until:2018-01-18', 'lacoste since:2018-01-18 until:2018-02-04', 'lacoste since:2018-02-04 until:2018-02-21', 'lacoste since:2018-02-21 until:2018-03-10', 'lacoste since:2018-03-10 until:2018-03-27', 'lacoste since:2018-03-27 until:2018-04-13', 'lacoste since:2018-04-13 until:2018-04-30', 'lacoste since:2018-04-30 until:2018-05-17', 'lacoste since:2018-05-17 until:2018-06-03', 'lacoste since:2018-06-03 until:2018-06-20', 'lacoste since:2018-06-20 until:2018-07-07', 'lacoste since:2018-07-07 until:2018-07-24', 'lacoste since:2018-07-24 until:2018-08-10', 'lacoste since:2018-08-10 until:2018-08-27', 'lacoste since:2018-08-27 until:2018-09-13', 'lacoste since:2018-09-13 until:2018-09-30', 'lacoste since:2018-09-30 until:2018-10-17', 'lacoste since:2018-10-17 until:2018-11-03', 'lacoste since:2018-11-03 until:2018-11-20', 'lacoste since:2018-11-20 until:2018-12-08']
INFO: Querying lacoste since:2018-01-18 until:2018-02-04
INFO: Querying lacoste sinc

In [117]:
with open(dump_file,"r") as f:
    dct = json.load(f)

print("Quantidade de meses considerados:  ", len(corpus))

Quantidade de meses considerados:   12


In [118]:

pp_corpus = []
for key in dct.keys():
    
    unit_tweets = []
    for txt in dct[key]:
        txt = word_tokenize(txt)
    
        #remove punctuation
        pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
        txt = list(filter(None, [pattern.sub('', token) for token in txt]))
    
        #lowercase
        txt = [x.lower() for x in txt]

        #remove stopwords
        sw = nltk.corpus.stopwords.words('portuguese')
        txt = [w for w in txt if w not in sw]
    
        unit_tweets.append(txt)
    pp_corpus.append(unit_tweets)
        
    

In [150]:
model_list = []
for unit in pp_corpus:
    model = Word2Vec(unit, size=100, window=5, min_count=1, workers=4)
    model_list.append(model)

In [152]:
for idx, model in enumerate(model_list):
    path = chosen_dir + 'word2vec_' + str(query[0]) + '_month_' + str(idx) + '.model'
    model.wv.save_word2vec_format(path)